In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from typing import List, Tuple
import os


class FactorModel(nn.Module):
    def __init__(self, N: int, K: List[int], d: int):
        super(FactorModel, self).__init__()
        self.N = N  # Number of questions
        self.K = K  # List of number of categorical answers for each question
        self.d = d  # Latent dimension
        self.W = nn.ModuleList([nn.Linear(d, K[i] - 1, bias=False) for i in range(N)])
        self.biases = nn.ParameterList(
            [nn.Parameter(torch.randn(K[i] - 1)) for i in range(N)]
        )

    def forward(self, v: torch.Tensor) -> List[torch.Tensor]:
        logits = [self.W[n](v) + self.biases[n] for n in range(self.N)]
        # Append zero logits for the last category to each question's logits
        logits = [
            torch.cat((logit, torch.zeros(logit.size(0), 1).to(logit.device)), dim=1)
            for logit in logits
        ]
        return logits

    def predict_proba(self, v: torch.Tensor) -> List[torch.Tensor]:
        logits = self.forward(v)  # List of logits for each question
        probabilities = [torch.softmax(logit, dim=-1) for logit in logits]
        return probabilities


def loss_function(
    model: FactorModel,
    v: torch.Tensor,
    answers: torch.Tensor,
    lambda1: float,
    lambda2: float,
) -> torch.Tensor:

    criterion = nn.NLLLoss(reduction="sum")
    logits = model.forward(v)

    total_loss = 0
    for n in range(model.N):
        log_probs = nn.functional.log_softmax(logits[n], dim=1)
        total_loss += criterion(log_probs, answers[:, n])
    nll_loss = total_loss.clone()
    # L2 Regularization
    l2_reg = sum(torch.norm(W.weight, 2) ** 2 for W in model.W)
    total_loss += lambda1 * l2_reg
    total_loss += lambda2 * torch.norm(v, 2) ** 2
    return total_loss, nll_loss


def train_model(
    model: FactorModel,
    answers: torch.Tensor,
    epochs: int = 1000,
    lr: float = 0.01,
    lambda1: float = 0.01,
    lambda2: float = 0.01,
) -> Tuple[FactorModel, torch.Tensor]:

    v = torch.randn(len(answers), model.d, requires_grad=True)

    # optimizer = optim.Adam(model.parameters(), lr=lr)
    optimizer = optim.Adam(list(model.parameters()) + [v], lr=lr)
    for epoch in range(epochs):
        optimizer.zero_grad()
        loss, nllloss = loss_function(model, v, answers, lambda1, lambda2)
        loss.backward()
        optimizer.step()
        # if epoch % 100 == 0:
        # print(f"Epoch {epoch}, Loss: {loss.item()}, nll_loss: {nllloss.item()}")
    print(
        f"d: {model.d}, Epoch {epoch}, Loss: {loss.item()}, nll_loss: {nllloss.item()}"
    )
    return model, v

In [33]:
import json

with open("../data/processed/narrative_questions.json") as f:
    data = json.load(f)
Q = data.values()
K = []
for q in Q:
    choice = q["choices"].values()
    K.append(len(choice))
print(K)

[3, 3, 3, 3, 3, 3]


In [34]:
import json

def extract_answers(file):
    with open(file) as f:
        data = json.load(f)

    # Initialize a list to store the extracted values
    answers = []

    # Iterate through the JSON objects to extract `post_LLM_answer`
    for item in data:

        if "Narrative_answers" in item:
            # get the value under the key `answers`
            answer = item["Narrative_answers"]
            answers.append(list(answer.values()))

    # convert the list of extracted values to a numpy array
    answers = np.array(answers, dtype=object)
    answers = np.where(answers == "A", 0, answers)
    answers = np.where(answers == "B", 1, answers)
    answers = np.where(answers == "C", 2, answers)
    print(answers)
    answers = answers.astype(int)
    return answers

In [35]:
answers = []
#for file in file_list:
answer = extract_answers("../data/processed/UN_data_with_narrative_answers.json")
print(answer.shape)
answers.append(answer)
answers = np.array(answers)

# reshape answers to 600,80
answers = answers.reshape(1138, 6)
answers.max()

[[2 1 2 2 2 2]
 [2 2 2 2 2 2]
 [2 2 2 2 2 2]
 ...
 [2 2 2 2 2 2]
 [2 1 2 2 2 2]
 [0 2 2 2 2 2]]
(1138, 6)


np.int64(2)

In [36]:
print(K)

[3, 3, 3, 3, 3, 3]


In [37]:
ans_max = list(answers.max(axis=0) + 1)
for i in range(len(ans_max)):
    if ans_max[i] > K[i]:
        print(i + 1, ans_max[i], K[i])

In [38]:
answers = torch.tensor(answers, dtype=torch.long)

In [39]:
N = len(K)  # Number of questions

M = answers.shape[0]  # Number of articles

In [40]:
epo = 20000

for d in [3]:
    model = FactorModel(N, K, d)
    trained_model, v = train_model(
        model, answers, epochs=epo, lr=0.01, lambda1=0.01, lambda2=0.01
    )
    # Extract the latent vectors v
    latent_vectors = v.detach().numpy()

d: 3, Epoch 19999, Loss: 382.9897155761719, nll_loss: 302.0203552246094


In [41]:
# Define your directory and filename
directory = "../data/processed/"  # Replace with your desired directory path
filename = f"latent_vector_{d}.csv"

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Construct the full file path
file_path = os.path.join(directory, filename)

# Save the latent vectors to the CSV file at the specified path
np.savetxt(
    file_path,
    latent_vectors,
    delimiter=",",
)